In [1]:
from sklearn import model_selection, datasets, svm
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import qiskit
import numpy as np
import copy
import matplotlib.pyplot as plt
from qiskit_aer import AerSimulator

In [23]:
N = 5

def feature_map(X) :
    q = QuantumRegister(N)
    c = ClassicalRegister(1)
    qc = QuantumCircuit(q,c)
    
    for i in range(0, len(X), 2):
        qc.rx(X[i], i // 2);
        qc.ry(X[i + 1], i // 2);
        
    return qc, c

In [24]:
def variational_circuit(qc, theta):
    
    for i in range(N-1):
        qc.cx(i, i+1)
    qc.cx(N-1, 0)
    for i in range(N):
        qc.ry(theta[i], i)
    return qc

In [25]:
def quantum_nn(X, theta, simulator=True):
    
    qc, c = feature_map(X)
    qc = variational_circuit(qc, theta)
    qc.measure(0,c)
    
    shots = 1E4
    backend = AerSimulator()
    job = backend.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts(qc)
    
    return(counts['1']/shots)

In [26]:
def loss (prediction, target):
    return (prediction - target)**2

In [27]:
def gradient(X, Y, theta):
    
    delta = 0.01
    grad = []
    for i in range(len(theta)):
        
        dtheta = copy.copy(theta)
        dtheta[i] += delta
        
        pred1 = quantum_nn(X, dtheta)
        pred2 = quantum_nn(X, theta)
        
        grad.append((loss(pred1, Y) - loss(pred2, Y)) / delta)
        
    return np.array(grad)

In [28]:
def accuracy(X, Y, theta):
    
    counter = 0
    for X_i, Y_i in zip(X, Y):
        
        prediction = quantum_nn(X_i, theta)
        
        if prediction < 0.5 and Y_i == 0:
            counter += 1
        elif prediction >= 0.5 and Y_i == 1:
            counter += 1
            
    return counter/len(Y)

In [30]:
normal_train_data = np.load('normal_train_data.npy')
normal_test_data = np.load('normal_test_data.npy')
anomalous_train_data = np.load('anomalous_train_data.npy')
anomalous_test_data = np.load('anomalous_test_data.npy')
train_data = np.load('train_data.npy')
test_data = np.load('test_data.npy')
train_labels = np.load('train_labels.npy')
test_labels = np.load('test_labels.npy')

In [34]:
eta = 0.05
loss_list = []
theta = np.ones(N)

t_d_1 = np.concatenate((normal_train_data[:500],anomalous_train_data[:500]))
y = np.concatenate((np.ones(500, dtype=int), np.zeros(500, dtype=int)))

from sklearn.decomposition import PCA
pca = PCA(n_components=10)  # Set number of components to 24
train_data_reduced = pca.fit_transform(t_d_1, y)
validation_data_reduced = pca.fit_transform(test_data, test_labels)

print('Epoch \t Loss \t Training Accuracy')

for i in range(20):
    
    loss_tmp = []
    for X_i, Y_i in zip(train_data_reduced, y):

        prediction = quantum_nn(X_i, theta)
        loss_tmp.append(loss(prediction, Y_i))
        theta = theta - eta * gradient(X_i, Y_i, theta)
        
    loss_list.append(np.mean(loss_tmp))
    acc = accuracy(train_data_reduced, y, theta)
    
    print(f'{i} \t {loss_list[-1]:.3f} \t {acc:.3f}')

Epoch 	 Loss 	 Training Accuracy


KeyboardInterrupt: 